In [1]:
import pandas as pd
import numpy as np
from haversine import haversine

In [2]:
df = pd.read_csv('4_summary.txt')

In [3]:
IDs = df['od_id'].values

In [4]:
IDs[30]

'20190425B01703727'

In [5]:
df1 = pd.read_csv('/home/heejung/Dropbox/2019 현대MNS ETA/04. 개발/id_based_parsing/id_{}.csv'.format(IDs[30]))

In [6]:
df2 = df1.loc[0]

In [7]:
df2['lngs'] = df1['lon'].values

/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/heejung/.local/lib/python3.6/site-packages/pandas/core/series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value


In [8]:
df2['lats'] = df1['lat'].values

/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
tot_time = df1.loc[df1.index[-1], 'track_sum_travel_time']
tot_dist = df1.loc[df1.index[-1], 'sum_len']
df2['time'] = tot_time / 10
df2['dist'] = tot_dist / 10000

/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
time = df1.loc[0, 'track_in_time']
date = df1.loc[0, "track_in_date"]
test = str(date) + str('%6d' %time)
test = test.replace(' ', '0')

In [11]:
test = pd.to_datetime(test)
weekID = test.weekday()
df2['weekID'] = weekID
df2['dateID'] = test.day
timeID = (test.hour * 60) + (test.minute) + int(test.second / 60)
df2['timeID'] = timeID

/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/heejung/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
df2 = df2.rename({'od_id': 'driverID'})

In [13]:
driver =df2['driverID']
driver = driver.replace('B', '')
driver = driver[12:]
df2['driverID'] = int(driver)

In [14]:
distgap = (df1['sum_len'] - df1['flink_len'])/10000

In [15]:
dist = []
for i in range(len(df1['lon'])):
    if(i < len(df1['lon']) - 1):
        dist.append(haversine((df1.loc[i,'lon'], df1.loc[i, 'lat']), (df1.loc[i+1, 'lon'], df1.loc[i+1, 'lat'])))
dist = pd.Series(dist)
flink_index = np.array(df1.index[df1['index'] == 0])
lis = np.array(df1.index)
idx_del = np.delete(lis, flink_index)
for i in idx_del:
    distgap[i] = distgap[i-1] +dist[i-1]

In [16]:
hour = (df1['point_in_time'] / 10000).astype(int)
minute = (df1['point_in_time'] / 100).astype(int) - hour*100
second = df1['point_in_time'].astype(int) - minute*100 - hour*10000
timegap = hour*60*60 + minute*60 +second
timegap = timegap - timegap[0]

In [17]:
df2['time_gap'] = timegap.values
df2['dist_gap'] = distgap.values
states = [1.0]*len(df2['lngs'])
df2['states']=states

In [18]:
df2 = df2.drop(['Unnamed: 0', 'lon', 'lat', 'index', 'point_in_time','seq', 'flink', 'tlink', 'turn', 'flink_len', 'sum_len', 'facil', 'road_type', 'link_type','light', 'lane', 'outlink_cnt', 'zone', 'track_in_stamp', 'track_in_date', 'track_in_time', 'track_speed', 'track_travel_time', 'track_sum_travel_time', 'static_spd', 'hq_speed', 'ol_rot_spd', 'ol_spd', 'svc_name'])

In [19]:
df2['lngs'] = df2['lngs'][::10]
df2['lats'] = df2['lats'][::10]
df2['time_gap'] = df2['time_gap'][::10]
df2['dist_gap'] = df2['dist_gap'][::10]
df2['states'] = df2['states'][::10]

In [20]:
df2

driverID                                                 3727
lngs        [127.03730555555559, 127.03841666666668, 127.0...
lats        [37.485, 37.48516666666666, 37.485333333333344...
time                                                   1220.9
dist                                                    6.313
weekID                                                      3
dateID                                                     25
timeID                                                   1273
time_gap    [0, 9, 19, 153, 165, 180, 211, 222, 237, 265, ...
dist_gap    [0.0, 0.10177996095529732, 0.21777996516501583...
states      [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
Name: 0, dtype: object

In [21]:
df2.to_json('data.txt')

In [22]:
len(df2['lngs'])

50